### 1. Imports

In [1]:
import pandas as pd
import inflection

#### 1.2 Loading Data

In [2]:
# Coletando os dados 
link_pagina = 'https://raw.githubusercontent.com/sthemonica/alura-voz/main/Dados/Telco-Customer-Churn.json'
df_json = pd.read_json(link_pagina)


### 2. Data Description

#### 2.1 Data Visualization
Após importar os dados, notamos que há informações aninhadas em formato json. Logo, temos que reestruturar para que fique tabulado corretamente, utilizaremos ferramentas nas próximas células para transformar neste o Dataframe final a ser explorado.

In [3]:
# Criando uma função para poupar linhas de código e resolver o problema de não estar tabulado corretamente
columns = ['customer', 'phone', 'internet', 'account']
for column in columns:
    df_temp = pd.json_normalize(data=df_json[column], sep='_')
    df_json = pd.concat([df_json, df_temp], axis=1)

In [4]:
# removendo as colunas com os formatos json
df_json.drop(columns=['customer', 'phone', 'internet', 'account'],inplace=True)

In [5]:
# removendo colunas duplicadas
df_json = df_json.loc[:,~df_json.columns.duplicated()]

In [6]:
# padronizando os headers para snake case
cols_snake = list( map( lambda x: inflection.underscore( x ), df_json.columns ) )
df_json.columns = cols_snake

In [7]:
# transformando valores vazios e ausentes em NA
df_json = df_json.mask(df_json == ' ').mask(df_json == '')

In [8]:
# removendo 11 linhas que estão com valores vazios em charges_total para converter em float
df_json.dropna(subset=['charges_total'], inplace=True)
df_json['charges_total'] = df_json['charges_total'].astype('float')

In [9]:
# Visualizando o cabeçalho
df_json.head()

,customer_id,churn,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service,...,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,contract,paperless_billing,payment_method,charges_monthly,charges_total
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,...,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.30
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,...,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.40
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,...,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.9,280.85
3,0011-IGKFF,Yes,Male,1,Yes,No,13,Yes,No,Fiber optic,...,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.0,1237.85
4,0013-EXCHZ,Yes,Female,1,Yes,No,3,Yes,No,Fiber optic,...,No,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.9,267.40


In [10]:
df_json.describe()

,senior_citizen,tenure,charges_monthly,charges_total
count,7256.0000,7256.000000,7256.000000,7256.000000
mean,0.1629,32.395535,64.755423,2280.634213
std,0.3693,24.558067,30.125739,2268.632997
min,0.0000,1.000000,18.250000,18.800000
25%,0.0000,9.000000,35.450000,400.225000
50%,0.0000,29.000000,70.300000,1391.000000
75%,0.0000,55.000000,89.900000,3785.300000
max,1.0000,72.000000,118.750000,8684.800000


In [11]:
df_json.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7256 entries, 0 to 7266
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   customer_id        7256 non-null   object 
 1   churn              7032 non-null   object 
 2   gender             7256 non-null   object 
 3   senior_citizen     7256 non-null   int64  
 4   partner            7256 non-null   object 
 5   dependents         7256 non-null   object 
 6   tenure             7256 non-null   int64  
 7   phone_service      7256 non-null   object 
 8   multiple_lines     7256 non-null   object 
 9   internet_service   7256 non-null   object 
 10  online_security    7256 non-null   object 
 11  online_backup      7256 non-null   object 
 12  device_protection  7256 non-null   object 
 13  tech_support       7256 non-null   object 
 14  streaming_tv       7256 non-null   object 
 15  streaming_movies   7256 non-null   object 
 16  contract           7256 

#### 2.2 Processing null values
Há pouco conseguimos ter acesso à todas as informações dos json aninhados, trataremos os valores nulos/ausentes.

In [12]:
df_json.isna().sum()

customer_id            0
churn                224
gender                 0
senior_citizen         0
partner                0
dependents             0
tenure                 0
phone_service          0
multiple_lines         0
internet_service       0
online_security        0
online_backup          0
device_protection      0
tech_support           0
streaming_tv           0
streaming_movies       0
contract               0
paperless_billing      0
payment_method         0
charges_monthly        0
charges_total          0
dtype: int64

In [13]:
# deletando as linhas com valores nulos em churn. Como é nossa variável resposta, não
# temos como inferir valores.
df_json.dropna(inplace=True)

In [14]:
df_json.isna().sum()

customer_id          0
churn                0
gender               0
senior_citizen       0
partner              0
dependents           0
tenure               0
phone_service        0
multiple_lines       0
internet_service     0
online_security      0
online_backup        0
device_protection    0
tech_support         0
streaming_tv         0
streaming_movies     0
contract             0
paperless_billing    0
payment_method       0
charges_monthly      0
charges_total        0
dtype: int64

#### 2.4 Discriptive Statistics
Agora iremos fazer uma análise estatística para encontrar valores negativos, outliers e etc.

##### Numerical Attributes

In [15]:
num_att = df_json.select_dtypes(['int64', 'float64'])
# getting skew
skew = pd.DataFrame(num_att.apply(lambda x: x.skew())).round(2).T
skew.rename(index={0: 'skew'}, inplace=True)

# getting kurtosis
kurtosis = pd.DataFrame(num_att.apply(lambda x: x.kurtosis())).round(2).T
kurtosis.rename(index={0: 'kurtosis'}, inplace=True)

# numerical features 
describe = df_json.describe().round(2)
m = pd.concat([describe, skew, kurtosis], axis=0)
m

,senior_citizen,tenure,charges_monthly,charges_total
count,7032.00,7032.00,7032.00,7032.00
mean,0.16,32.42,64.80,2283.30
std,0.37,24.55,30.09,2266.77
min,0.00,1.00,18.25,18.80
25%,0.00,9.00,35.59,401.45
50%,0.00,29.00,70.35,1397.48
75%,0.00,55.00,89.86,3794.74
max,1.00,72.00,118.75,8684.80
skew,1.83,0.24,-0.22,0.96
kurtosis,1.35,-1.39,-1.26,-0.23


##### Categorical Attributes

In [16]:
cat_att = df_json.select_dtypes(exclude=['int64', 'float64'])
cat_att.apply( lambda x: x.unique().shape[0] )

customer_id          7032
churn                   2
gender                  2
partner                 2
dependents              2
phone_service           2
multiple_lines          3
internet_service        3
online_security         3
online_backup           3
device_protection       3
tech_support            3
streaming_tv            3
streaming_movies        3
contract                3
paperless_billing       2
payment_method          4
dtype: int64

In [17]:
for column in df_json.columns:
    print(df_json[column].unique())

['0002-ORFBO' '0003-MKNFE' '0004-TLHLJ' ... '9992-UJOEL' '9993-LHIEB'
 '9995-HOTOH']
['No' 'Yes']
['Female' 'Male']
[0 1]
['Yes' 'No']
['Yes' 'No']
[ 9  4 13  3 71 63  7 65 54 72  5 56 34  1 45 50 23 55 26 69 37 49 66 67
 20 43 59 12 27  2 25 29 14 35 64 39 40 11  6 30 70 57 58 16 32 33 10 21
 61 15 44 22 24 19 47 62 46 52  8 60 48 28 41 53 68 31 36 17 18 51 38 42]
['Yes' 'No']
['No' 'Yes' 'No phone service']
['DSL' 'Fiber optic' 'No']
['No' 'Yes' 'No internet service']
['Yes' 'No' 'No internet service']
['No' 'Yes' 'No internet service']
['Yes' 'No' 'No internet service']
['Yes' 'No' 'No internet service']
['No' 'Yes' 'No internet service']
['One year' 'Month-to-month' 'Two year']
['Yes' 'No']
['Mailed check' 'Electronic check' 'Credit card (automatic)'
 'Bank transfer (automatic)']
[65.6  59.9  73.9  ... 91.75 68.8  67.85]
[ 593.3   542.4   280.85 ...  742.9  4627.65 3707.6 ]


### 3. Feature Engineering
Primeira manipulação das variáveis, antecipadamente iremos apenas mudar os nomes das colunas.

In [18]:
# Padronizando valores no 
for column in df_json.columns[8:16]:
    df_json.loc[:,column] = df_json[column].apply(lambda x: 'No' if 'No' in x else x)

# criando feature de gasto diário partindo do charges_total
df_json.insert(loc=18, column='daily_charge', value=df_json['charges_total']/(df_json['tenure']*30))

In [19]:


df_json.rename(columns = {'churn':'target',
           'customer_id':'ID', 
           'gender':'GENERO', 
           'senior_citizen':'IDOSO', 
           'partner':'CÔNJUGE',
           'dependents':'DEPENDENTES', 
           'tenure':'MESES_CONTRATO', 
           'phone_service':'ASS_TEL', 
           'multiple_lines':'ASS_MULTI_TEL', 
           'internet_service':'ASS_INTERNET', 
           'online_security':'ASS_AD_SEGURNAÇA',
           'online_backup':'BACKUP', 
           'device_protection':'PROTEÇÃO_DISPOSITIVO', 
           'tech_support':'SUP_TÉCNICO',
           'streaming_tv':'TV_CABO', 
           'streaming_movies':'STREAMING', 
           'contract':'CONTRATO',
           'paperless_billing':'FATURA_ONLINE', 
           'payment_method':'FORMA_PAGAMENTO', 
           'charges_monthly':'GASTO_MENSAL',
           'charges_total':'GASTO_TOTAL', 
           'daily_charge':'GASTO_DIARIO'}, inplace=True
)

In [20]:
df_json.head()

,ID,target,GENERO,IDOSO,CÔNJUGE,DEPENDENTES,MESES_CONTRATO,ASS_TEL,ASS_MULTI_TEL,ASS_INTERNET,...,PROTEÇÃO_DISPOSITIVO,SUP_TÉCNICO,TV_CABO,STREAMING,CONTRATO,FATURA_ONLINE,GASTO_DIARIO,FORMA_PAGAMENTO,GASTO_MENSAL,GASTO_TOTAL
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,...,No,Yes,Yes,No,One year,Yes,2.197407,Mailed check,65.6,593.30
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,...,No,No,No,Yes,Month-to-month,No,2.008889,Mailed check,59.9,542.40
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,...,Yes,No,No,No,Month-to-month,Yes,2.340417,Electronic check,73.9,280.85
3,0011-IGKFF,Yes,Male,1,Yes,No,13,Yes,No,Fiber optic,...,Yes,No,Yes,Yes,Month-to-month,Yes,3.173974,Electronic check,98.0,1237.85
4,0013-EXCHZ,Yes,Female,1,Yes,No,3,Yes,No,Fiber optic,...,No,Yes,Yes,No,Month-to-month,Yes,2.971111,Mailed check,83.9,267.40


### 4. Salvando o DataFrame em csv
Com o objetivo de futuras manipulações e análises. Por fim criar um modelo que seja adequado e consiga trazer resultados.

In [21]:
df_json.to_csv('E:\Alura_Challenge_Data_Science\data\df_limpo.csv')